# Twitter Crawler

In [10]:
import sys
!{sys.executable} -m pip install pymongo

    100% |████████████████████████████████| 348kB 2.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
# Import Packages
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pymongo
import numpy as np
import pandas as pd
import csv
from datetime import datetime

In [2]:
# Target URL
url = "https://www.twitter.com/Nike"

In [3]:
# Using Chrome WebDriver
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [4]:
# Beautiful Soup scrolls down to the bottom of the page
lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight

soupfetch = BeautifulSoup(driver.page_source, "html.parser")

In [22]:
# Open/Create a file to append data
csvFile = open('niketweets.csv', 'w+')
# Use csv Writer
csvWriter = csv.writer(csvFile)

In [23]:
# Initializing Parameters
tweet = []
hashtag = []
mentions = []
reply = []
like = [] 
retweet = []
date = []
time = []

In [24]:
# Fetching Twitter Data
row_count = 0
twitterwebdata = soupfetch.find_all('div', {'class': 'content'})
for twitter_tweet in twitterwebdata:
    try:
        tweet = twitter_tweet.find('p', {'class': 'TweetTextSize'}).text
        print(tweet)
    except:
        pass
    
    hashtag = []
    for tag in twitter_tweet.find_all('a', {'class': 'twitter-hashtag'}):
        hashtag.append(tag.text[1:])
    if hashtag == []:
        hashtag = np.NaN
    #print(hashtag)
    
    mentions = []
    for a in twitter_tweet.find_all('a', {'class': 'twitter-atreply'}):
        try:
            mentions.append(a['href'][1:])
        except:
            pass
    if mentions == []:
        mentions = np.NaN
    #print(mentions)
    
    thumbs = twitter_tweet.find_all('span', \
                        {'class': 'ProfileTweet-actionCountForPresentation'})
    try:
        reply = thumbs[0].text
        if reply[-1] == 'K':
            reply = int(float(reply[:-1]) * 1000)
        else:
            reply = int(float(reply.encode))
    except:
        pass
    #print(re)
    
    try:
        like = thumbs[3].text
        if like[-1] == 'K':
            like = int(float(like[:-1])*1000)
        else:
            like = int(float(like))
    except:
        pass
    #print(like)
    
    try:
        retweet = thumbs[2].text
        if retweet[-1] == 'K':
            retweet = int(float(retweet[:-1])*1000)
        else:
            retweet = int(float(retweet))
    except:
        pass
    #print(retweet)
    try:
        date = twitter_tweet.find("a",attrs={"class":"tweet-timestamp js-permalink js-nav js-tooltip"})["title"].split('-')[1]
    except:
        date = None
        pass
    
    try:
        time = twitter_tweet.find("a",attrs={"class":"tweet-timestamp js-permalink js-nav js-tooltip"})["title"].split('-')[0]
    except:
        time = None
        pass
    #print(time)
    #print(tweet, hashtag, mentions, reply, like, retweet, date, time)
    if all(attributes is not None for attributes in [tweet, date, time]):
        csvWriter.writerow([tweet, hashtag, mentions, reply, like, retweet, date, time])        

Don’t ask if your dreams are crazy. Ask if they’re crazy enough. #justdoitpic.twitter.com/Wd5L42egV8
He’s done it again. 

Never stop chasing your crazy dreams. #justdoitpic.twitter.com/sVXo12E1r9
Yesterday, 2:01:39 seemed crazy. Today, it’s the marathon world record. #justdoitpic.twitter.com/dm7JeOCPqN
Don't just be the fastest marathon runner in the world. Be the fastest in history. #justdoit @EliudKipchogepic.twitter.com/jGcoNy3Wsy
Yeah, that’s more like it. #justdoit
If you’re a girl from Compton, don’t just become a tennis player.
Dream of being the greatest athlete ever.
If you have only one hand, don’t just watch football, play it. At the highest level.
When they talk about the greatest team in the history of the sport, make sure it’s your team.
Believe in something, even if it means sacrificing everything.
Don’t become the best basketball player on the planet. Be bigger than basketball.
If you’re born a refugee, don’t let it stop you from playing soccer for the national team — 

A day has enough hours. You just have too many excuses.
Back to the blueprint. #airmaxhttps://amp.twimg.com/v/0449c772-b337-475b-9a4d-a020dc7382a4 …
There is no way to return the serve if you are not facing it. #serenareturns @serenawilliamspic.twitter.com/QVVYS7UjVT
Shopping your Instagram favorites just got easier. The Nike Women’s Instagram shop is here. http://gonike.me/6016FAxn pic.twitter.com/7HcrYPZgoL
You have to get good before you are good. #betterforitpic.twitter.com/pv7z19tzKt
Today is a starting block. Not a stumbling block. #justdoit
We all sweat in the same language. #womensday
Your body will thank you later. #justdoit
Today is up to you. #justdoit
The stars of tomorrow focus on today. #justdoit
Born in 1982. 3D printed in 2015. Introducing the first AF1 figurine, made with love at the #zoomcity bank. #nikeairhttps://amp.twimg.com/v/dcca00d3-daa3-4b64-a874-983f42aaf987 …
Above the rim. Beyond comparison. @ZachLaVine #NBAAllStarNYCpic.twitter.com/pkcAy5xxLd
A court fit fo

#justdoit RT @NikeFuel Summer means time for more.pic.twitter.com/zQfY4ym6Fo
Arrive early. Start fast. #futuregetsfasterpic.twitter.com/u8QlYRTLF9
Determination determines everything. #justdoit
Champion is the only label we'll remember.pic.twitter.com/t6VhYCc4Xc
No risk. No revenge. #riskeverythingpic.twitter.com/hjkwNLj0hW
If you are not ready to risk everything, there will be no revenge. Zlatan has spoken.https://amp.twimg.com/v/012951bf-8693-4c7d-86b2-2b3d843a9b4c …
Teach the world what your country taught you. #justdoitpic.twitter.com/3ZmKSVFMVK
Today a summer of risk starts. Zlatan is here to answer your questions. Go on, #AskZlatan.https://amp.twimg.com/v/461b5db6-6422-40e1-9f57-f2581e81fd07 …
Play every game like it's 'The Last Game.' #riskeverythinghttps://amp.twimg.com/v/b108fb9d-5d87-4041-8100-29025b60a61d …
There's no greater danger than playing it safe. http://gonike.me/6016ceQ2 
If you don't take risks, you'll always have regret. #justdoit
For the king of Paris, it’s an en

Resolutions aren’t supposed to be recycled.
The greatest resolutions are the ones you keep. http://nike.com/justdoit 
Want to continue helping today's kids get moving? Build on what you accomplished on 12.21 and visit http://www.designedtomove.org  to learn more.
It may be the darkest day of the year, but you still have 24 hours. Make it matter: http://gonike.me/1221 
Tomorrow, pledge your Nike+ activity to one of 10 programs that are helping to get kids active: http://gonike.me/1221 
Every mile counts on 12.21. Hit the road for a great cause and everyone wins. #flashrun http://gonike.me/1221 
On Saturday, our @NikeFuel counts more. I'm gettin after 7k. Join me, move for good: http://gonike.me/1221  #justdoit #teamnike
There's never enough time in the day, so let's make what you do on the longest night of the year matter more: http://gonike.me/1221 
If he never came back to the game.

He'd be a living legend.

But he wouldn't be @KobeBryant. #justdoit
http://youtu.be/ikC5GIpNFWw 
When 

# Analysis

In [25]:
# Loading Comma Seperated Data using read_table pandas function in 'nike_data' dataframe
nike_data = pd.read_table("niketweets.csv", sep=",", header=None)

In [26]:
# Adding Headers to Data
nike_data.columns = ["tweet", "hashtag", "mentions", "reply", "like", "retweet", "date", "time"]

In [27]:
nike_data

,tweet,hashtag,mentions,reply,like,retweet,date,time
0,Don’t ask if your dreams are crazy. Ask if the...,['justdoit'],NaN,16000.0,522000,228000,5 Sep 2018,10:09 AM
1,He’s done it again. \n\nNever stop chasing you...,['justdoit'],NaN,257.0,31000,6000,23 Sep 2018,5:55 PM
2,"Yesterday, 2:01:39 seemed crazy. Today, it’s t...",['justdoit'],NaN,214.0,29000,8900,16 Sep 2018,3:56 AM
3,Don't just be the fastest marathon runner in t...,['justdoit'],['EliudKipchoge'],104.0,6100,2600,15 Sep 2018,8:52 AM
4,"Yeah, that’s more like it. #justdoit",['justdoit'],NaN,1200.0,26000,5300,5 Sep 2018,10:14 AM
5,"If you’re a girl from Compton, don’t just beco...",NaN,NaN,310.0,19000,3600,5 Sep 2018,10:14 AM
6,"If you have only one hand, don’t just watch fo...",NaN,NaN,194.0,11000,2000,5 Sep 2018,10:14 AM
7,When they talk about the greatest team in the ...,NaN,NaN,168.0,11000,2400,5 Sep 2018,10:14 AM
8,"Believe in something, even if it means sacrifi...",NaN,NaN,1200.0,36000,17000,5 Sep 2018,10:14 AM
9,Don’t become the best basketball player on the...,NaN,NaN,84.0,12000,2500,5 Sep 2018,10:14 AM
